# Agenti Azure AI con supporto per il Protocollo di Contesto del Modello (MCP)

Questo notebook dimostra come utilizzare gli Agenti Azure AI con strumenti del Protocollo di Contesto del Modello (MCP) per creare un agente intelligente che può sfruttare server MCP esterni per capacità avanzate.


## Installare i pacchetti NuGet richiesti

Per prima cosa, dobbiamo installare il pacchetto Azure AI Agents Persistent, che fornisce la funzionalità principale per lavorare con Azure AI Agents.


## Vantaggi dell'autenticazione senza chiavi

Questo notebook dimostra **l'autenticazione senza chiavi**, che offre diversi vantaggi:
- ✅ **Nessuna gestione di chiavi API** - Utilizza l'autenticazione basata sull'identità di Azure
- ✅ **Sicurezza migliorata** - Nessun segreto memorizzato nel codice o nella configurazione
- ✅ **Rotazione automatica delle credenziali** - Azure gestisce il ciclo di vita delle credenziali
- ✅ **Accesso basato sui ruoli** - Utilizza Azure RBAC per permessi dettagliati

`DefaultAzureCredential` utilizzerà automaticamente la migliore fonte di credenziali disponibile:
1. Identità gestita (quando si esegue in Azure)
2. Credenziali Azure CLI (durante lo sviluppo)
3. Credenziali di Visual Studio
4. Variabili d'ambiente (se configurate)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Installa il pacchetto Azure Identity per l'autenticazione con i servizi Azure utilizzando DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importa i Namespace Necessari

Importa i namespace necessari per Azure AI Agents e Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Configurare Azure AI Agent Client (Autenticazione senza chiave)

Configura le variabili di configurazione e crea il PersistentAgentsClient utilizzando **l'autenticazione senza chiave**:
- **projectEndpoint**: L'endpoint del progetto Azure AI Foundry
- **modelDeploymentName**: Il nome del modello AI distribuito (GPT-4.1 nano)
- **mcpServerUrl**: L'URL del server MCP (Microsoft Learn API)
- **mcpServerLabel**: Un'etichetta per identificare il server MCP
- **DefaultAzureCredential**: Utilizza identità gestite, Azure CLI o altre fonti di credenziali (non sono richieste chiavi API)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Creare la definizione dello strumento MCP

Crea una definizione dello strumento MCP che si collega al server MCP di Microsoft Learn. Questo consentirà all'agente di accedere ai contenuti e alla documentazione di Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Creare l'agente AI

Crea un agente AI persistente con il modello specificato e gli strumenti MCP. L'agente è configurato con:
- Il modello GPT-4.1 nano
- Istruzioni per utilizzare gli strumenti MCP per assistenza
- Accesso al server Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Creare un Thread e Inviare un Messaggio

Crea un thread di conversazione e invia un messaggio all'utente chiedendo la differenza tra Azure OpenAI e OpenAI. Questo testerà la capacità dell'agente di utilizzare gli strumenti MCP per fornire informazioni accurate.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Configurare le risorse dello strumento MCP (Senza chiavi)

Configura le risorse dello strumento MCP. Per un approccio realmente senza chiavi, puoi rimuovere le intestazioni personalizzate se il server MCP supporta l'autenticazione basata sull'identità di Azure. L'esempio seguente mostra come aggiungere intestazioni se necessario, ma per scenari senza chiavi, queste potrebbero non essere richieste.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Avvia l'esecuzione dell'agente

Crea e avvia un'esecuzione per elaborare il messaggio dell'utente. L'agente utilizzerà gli strumenti e le risorse MCP configurati per generare una risposta.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Monitorare l'Esecuzione e Gestire le Approvazioni degli Strumenti (Senza Chiavi)

Monitora lo stato di esecuzione dell'agente e gestisce eventuali approvazioni degli strumenti necessarie. Questo ciclo:
1. Attende che l'esecuzione sia completata o richieda un'azione
2. Approva automaticamente le chiamate agli strumenti MCP quando necessario
3. Per l'autenticazione senza chiavi, le intestazioni potrebbero non essere necessarie se il server MCP supporta l'identità Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Visualizzare i Risultati della Conversazione

Recupera e mostra tutti i messaggi nella discussione, includendo sia la domanda dell'utente che la risposta dell'agente. I messaggi vengono visualizzati in ordine cronologico con indicazioni di timestamp e ruolo.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
